In [ ]:
pip install scmap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install structure-tensor

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for structure-tensor: filename=structure_tensor-0.2.0-py3-none-any.whl size=15924 sha256=0bc4a7d4bbe1cf9751cf242897cdd5283793f0d50a9e7b82c5a191ac04abca5b
  Stored in directory: /root/.cache/pip/wheels/91/20/c3/7f70390148b5bd4ca9efcdd82ba1c678e7d26194a1c4d9401d
Successfully built structure-tensor


In [ ]:
import torch
from tqdm import tqdm
import torch.nn as nn
import random
import skimage.io
import numpy as np
import torch.nn.functional as F
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from torch.utils.data import TensorDataset, DataLoader

import time
import numpy as np
import os


os.chdir('/content/drive/MyDrive/NyDLProjekt')

from structure_tensor import eig_special_3d, structure_tensor_3d
from data_funcs import DataPrepAndSave, CreateDataLoaders
from sympy import *
import numpy as np
from numpy import arctan2, sqrt
import numexpr as ne


def cords2angles(y):
    batch_size,c,height,width,depth = y.shape
    theta = torch.empty([batch_size,2,height,width,depth])
    theta = theta.cuda()
    theta[:,0,:,:,:] = torch.arctan2(y[:,1,:,:,:], y[:,0,:,:,:])
    theta[:,1,:,:,:] = torch.arctan2(torch.sqrt(y[:,0,:,:,:]**2+y[:,1,:,:,:]**2), y[:,2,:,:,:])
    return theta

def angles2cords(theta):
    batch_size,c,height,width,depth = theta.shape
    ynew = torch.empty([batch_size,3,height,width,depth])
    ynew = ynew.cuda()
    ynew[:,0,:,:,:] = torch.cos(theta[:,0,:,:,:]) * torch.sin(theta[:,1,:,:,:])#x
    ynew[:,1,:,:,:] = torch.sin(theta[:,1,:,:,:]) * torch.sin(theta[:,0,:,:,:]) #y
    ynew[:,2,:,:,:] = torch.cos(theta[:,1,:,:,:]) #z
    return ynew

def anglediff(y,y1):
    delta = 2*(1-torch.cos(y1-y))
    return torch.mean(delta)

In [ ]:
class Net_1(nn.Module):
    def __init__(self):
        super(Net_1, self).__init__()
        self.out1 = 6*2
        self.out2 = 12*2
        self.out3 = 24*2
        self.out4 = 48*2
        self.out5 = 96*2


        self.kernel_size = 3
        self.padding = 1


        #StructureTensorPart
        self.up = nn.Upsample(scale_factor = 2, mode = 'trilinear',align_corners=True)
        self.down = nn.Upsample(scale_factor = 1/2,mode = 'trilinear',align_corners=True)
        self.down2 = nn.Upsample(scale_factor = 24/25,mode = 'trilinear',align_corners=True)
        self.S_conv1 = nn.Conv3d(in_channels = 3, out_channels = self.out1, kernel_size = self.kernel_size, padding = 0 )
        self.S_conv2 = nn.Conv3d(in_channels = self.out1, out_channels = self.out1, kernel_size =self.kernel_size, padding = self.padding)
        self.Batchnorm1S = nn.BatchNorm3d(self.out1)
        self.S_mp1 = nn.MaxPool3d(kernel_size = 2)
        self.S_conv3 = nn.Conv3d(in_channels = self.out1, out_channels = self.out2, kernel_size =self.kernel_size, padding = self.padding)
        self.S_conv4 = nn.Conv3d(in_channels = self.out2, out_channels = self.out2, kernel_size =self.kernel_size, padding = self.padding)
        self.Batchnorm2S = nn.BatchNorm3d(self.out2)

        self.S_mp2 = nn.MaxPool3d(kernel_size = 2)
        self.S_conv5 = nn.Conv3d(in_channels = self.out2, out_channels = self.out3, kernel_size = self.kernel_size, padding = self.padding)
        self.S_conv6 = nn.Conv3d(in_channels = self.out3, out_channels = self.out3, kernel_size = self.kernel_size, padding = self.padding)
        self.Batchnorm3S = nn.BatchNorm3d(self.out3)

        self.S_mp3 = nn.MaxPool3d(kernel_size = 2)
        self.S_conv7 = nn.Conv3d(in_channels = self.out3, out_channels = self.out4, kernel_size = self.kernel_size, padding = self.padding)
        self.S_conv8 = nn.Conv3d(in_channels = self.out4, out_channels = self.out4, kernel_size = self.kernel_size, padding = self.padding)

        self.Batchnorm4S = nn.BatchNorm3d(self.out4)
        self.S_mp4 = nn.MaxPool3d(kernel_size = 2)
        self.S_conv9 = nn.Conv3d(in_channels = self.out4, out_channels = self.out5, kernel_size = self.kernel_size, padding = self.padding)
        self.S_conv10 = nn.Conv3d(in_channels = self.out5, out_channels = self.out5, kernel_size = self.kernel_size, padding = self.padding)
        


        #FullImagePart
        self.I_conv1 = nn.Conv3d(in_channels = 1, out_channels = self.out1, kernel_size = self.kernel_size, padding = 0 )
        self.I_conv2 = nn.Conv3d(in_channels = self.out1, out_channels = self.out1, kernel_size =self.kernel_size, padding = self.padding)
        self.Batchnorm1I = nn.BatchNorm3d(self.out1)

        self.I_mp1 = nn.MaxPool3d(kernel_size = 2)
        self.I_conv3 = nn.Conv3d(in_channels = self.out1, out_channels = self.out2, kernel_size =self.kernel_size, padding = self.padding)
        self.I_conv4 = nn.Conv3d(in_channels = self.out2, out_channels = self.out2, kernel_size =self.kernel_size, padding = self.padding)
        self.Batchnorm2I = nn.BatchNorm3d(self.out2)
        self.I_mp2 = nn.MaxPool3d(kernel_size = 2)
        self.I_conv5 = nn.Conv3d(in_channels = self.out2, out_channels = self.out3, kernel_size = self.kernel_size, padding = self.padding)
        self.I_conv6 = nn.Conv3d(in_channels = self.out3, out_channels = self.out3, kernel_size = self.kernel_size, padding = self.padding)
        self.Batchnorm3I = nn.BatchNorm3d(self.out3)
        self.I_mp3 = nn.MaxPool3d(kernel_size = 2)
        self.I_conv7 = nn.Conv3d(in_channels = self.out3, out_channels = self.out4, kernel_size = self.kernel_size, padding = self.padding)
        self.I_conv8 = nn.Conv3d(in_channels = self.out4, out_channels = self.out4, kernel_size = self.kernel_size, padding = self.padding)
        self.Batchnorm4I = nn.BatchNorm3d(self.out4)
        self.I_mp4 = nn.MaxPool3d(kernel_size = 2)
        self.I_conv9 = nn.Conv3d(in_channels = self.out4, out_channels = self.out5, kernel_size = self.kernel_size, padding = self.padding)
        self.I_conv10 = nn.Conv3d(in_channels = self.out5, out_channels = self.out5, kernel_size = self.kernel_size, padding = self.padding)
        



        #Combining part

        self.convST1 = nn.ConvTranspose3d(in_channels = self.out5, out_channels = self.out4,kernel_size = 2, stride = 2)
        self.convIT1 = nn.ConvTranspose3d(in_channels = self.out5, out_channels = self.out4,kernel_size = 2, stride = 2)

        self.conv7 = nn.Conv3d(in_channels = 4*self.out4, out_channels = self.out4, kernel_size =self.kernel_size, padding = self.padding)
        self.conv8 = nn.Conv3d(in_channels = self.out4, out_channels = self.out4, kernel_size =self.kernel_size, padding = self.padding)
        
        
        self.convT2 = nn.ConvTranspose3d(in_channels = self.out4, out_channels = self.out3,kernel_size = 2,stride = 2)


        self.conv9 = nn.Conv3d(in_channels = 3 * self.out3, out_channels = self.out3, kernel_size =self.kernel_size, padding = self.padding)
        self.conv10 = nn.Conv3d(in_channels = self.out3, out_channels = self.out3, kernel_size =self.kernel_size, padding = self.padding)
        
        self.convT3 = nn.ConvTranspose3d(in_channels = self.out3, out_channels = self.out2, kernel_size = 2, stride = 2)
        
        self.conv11 = nn.Conv3d(in_channels = 3 * self.out2, out_channels = self.out2, kernel_size =self.kernel_size, padding = self.padding)
        self.conv12 = nn.Conv3d(in_channels = self.out2, out_channels = self.out2, kernel_size =self.kernel_size, padding = self.padding)
        
        self.convT4 = nn.ConvTranspose3d(in_channels = self.out2, out_channels = self.out1,kernel_size = 2,stride = 2)
        self.convT5 = nn.ConvTranspose3d(in_channels = 3*self.out1, out_channels = self.out1, kernel_size =self.kernel_size, stride = 1, padding = 0)
        
        self.conv13 = nn.Conv3d(in_channels = self.out1, out_channels = self.out1, kernel_size =self.kernel_size, padding = self.padding)
        self.conv14 = nn.Conv3d(in_channels = self.out1, out_channels = self.out1, kernel_size =self.kernel_size, padding = self.padding)

        self.conv15 = nn.Conv3d(in_channels = self.out1, out_channels = 3, kernel_size =self.kernel_size, padding = self.padding)


    def st_vec(self, x, scale):
      X = torch.zeros([batch_size,3,int(height*scale),int(width*scale),int(depth*scale)])
      X.cuda()
      for b in range(batch_size):
          s = structure_tensor_3d(x[b,0,:,:,:],1/2,1)
          val,vec = eig_special_3d(s)
          X[b,:,:,:,:] = torch.Tensor(vec)
      return X



    def forward(self, x):
       xstruct = self.down(x)
       s2 = self.st_vec(xstruct, scale)
       s1 = self.up(s2)
       u1 = F.normalize(s1)
       return s1


class Net_2(nn.Module):
    def __init__(self):
        super(Net_2, self).__init__()
        self.out1 = 6*2
        self.out2 = 12*2
        self.out3 = 24*2
        self.out4 = 48*2
        self.out5 = 96*2


        self.kernel_size = 3
        self.padding = 1


        #StructureTensorPart
        self.up = nn.Upsample(scale_factor = 2, mode = 'trilinear',align_corners=True)
        self.down = nn.Upsample(scale_factor = 1/2,mode = 'trilinear',align_corners=True)
        self.down2 = nn.Upsample(scale_factor = 24/25,mode = 'trilinear',align_corners=True)
        self.S_conv1 = nn.Conv3d(in_channels = 3, out_channels = self.out1, kernel_size = self.kernel_size, padding = 0 )
        self.S_conv2 = nn.Conv3d(in_channels = self.out1, out_channels = self.out1, kernel_size =self.kernel_size, padding = self.padding)
        self.Batchnorm1S = nn.BatchNorm3d(self.out1)
        self.S_mp1 = nn.MaxPool3d(kernel_size = 2)
        self.S_conv3 = nn.Conv3d(in_channels = self.out1, out_channels = self.out2, kernel_size =self.kernel_size, padding = self.padding)
        self.S_conv4 = nn.Conv3d(in_channels = self.out2, out_channels = self.out2, kernel_size =self.kernel_size, padding = self.padding)
        self.Batchnorm2S = nn.BatchNorm3d(self.out2)

        self.S_mp2 = nn.MaxPool3d(kernel_size = 2)
        self.S_conv5 = nn.Conv3d(in_channels = self.out2, out_channels = self.out3, kernel_size = self.kernel_size, padding = self.padding)
        self.S_conv6 = nn.Conv3d(in_channels = self.out3, out_channels = self.out3, kernel_size = self.kernel_size, padding = self.padding)
        self.Batchnorm3S = nn.BatchNorm3d(self.out3)

        self.S_mp3 = nn.MaxPool3d(kernel_size = 2)
        self.S_conv7 = nn.Conv3d(in_channels = self.out3, out_channels = self.out4, kernel_size = self.kernel_size, padding = self.padding)
        self.S_conv8 = nn.Conv3d(in_channels = self.out4, out_channels = self.out4, kernel_size = self.kernel_size, padding = self.padding)

        self.Batchnorm4S = nn.BatchNorm3d(self.out4)
        self.S_mp4 = nn.MaxPool3d(kernel_size = 2)
        self.S_conv9 = nn.Conv3d(in_channels = self.out4, out_channels = self.out5, kernel_size = self.kernel_size, padding = self.padding)
        self.S_conv10 = nn.Conv3d(in_channels = self.out5, out_channels = self.out5, kernel_size = self.kernel_size, padding = self.padding)
        


        #FullImagePart
        self.I_conv1 = nn.Conv3d(in_channels = 1, out_channels = self.out1, kernel_size = self.kernel_size, padding = 0 )
        self.I_conv2 = nn.Conv3d(in_channels = self.out1, out_channels = self.out1, kernel_size =self.kernel_size, padding = self.padding)
        self.Batchnorm1I = nn.BatchNorm3d(self.out1)

        self.I_mp1 = nn.MaxPool3d(kernel_size = 2)
        self.I_conv3 = nn.Conv3d(in_channels = self.out1, out_channels = self.out2, kernel_size =self.kernel_size, padding = self.padding)
        self.I_conv4 = nn.Conv3d(in_channels = self.out2, out_channels = self.out2, kernel_size =self.kernel_size, padding = self.padding)
        self.Batchnorm2I = nn.BatchNorm3d(self.out2)
        self.I_mp2 = nn.MaxPool3d(kernel_size = 2)
        self.I_conv5 = nn.Conv3d(in_channels = self.out2, out_channels = self.out3, kernel_size = self.kernel_size, padding = self.padding)
        self.I_conv6 = nn.Conv3d(in_channels = self.out3, out_channels = self.out3, kernel_size = self.kernel_size, padding = self.padding)
        self.Batchnorm3I = nn.BatchNorm3d(self.out3)
        self.I_mp3 = nn.MaxPool3d(kernel_size = 2)
        self.I_conv7 = nn.Conv3d(in_channels = self.out3, out_channels = self.out4, kernel_size = self.kernel_size, padding = self.padding)
        self.I_conv8 = nn.Conv3d(in_channels = self.out4, out_channels = self.out4, kernel_size = self.kernel_size, padding = self.padding)
        self.Batchnorm4I = nn.BatchNorm3d(self.out4)
        self.I_mp4 = nn.MaxPool3d(kernel_size = 2)
        self.I_conv9 = nn.Conv3d(in_channels = self.out4, out_channels = self.out5, kernel_size = self.kernel_size, padding = self.padding)
        self.I_conv10 = nn.Conv3d(in_channels = self.out5, out_channels = self.out5, kernel_size = self.kernel_size, padding = self.padding)
        



        #Combining part

        self.convST1 = nn.ConvTranspose3d(in_channels = self.out5, out_channels = self.out4,kernel_size = 2, stride = 2)
        self.convIT1 = nn.ConvTranspose3d(in_channels = self.out5, out_channels = self.out4,kernel_size = 2, stride = 2)

        self.conv7 = nn.Conv3d(in_channels = 4*self.out4, out_channels = self.out4, kernel_size =self.kernel_size, padding = self.padding)
        self.conv8 = nn.Conv3d(in_channels = self.out4, out_channels = self.out4, kernel_size =self.kernel_size, padding = self.padding)
        
        
        self.convT2 = nn.ConvTranspose3d(in_channels = self.out4, out_channels = self.out3,kernel_size = 2,stride = 2)


        self.conv9 = nn.Conv3d(in_channels = 3 * self.out3, out_channels = self.out3, kernel_size =self.kernel_size, padding = self.padding)
        self.conv10 = nn.Conv3d(in_channels = self.out3, out_channels = self.out3, kernel_size =self.kernel_size, padding = self.padding)
        
        self.convT3 = nn.ConvTranspose3d(in_channels = self.out3, out_channels = self.out2, kernel_size = 2, stride = 2)
        
        self.conv11 = nn.Conv3d(in_channels = 3 * self.out2, out_channels = self.out2, kernel_size =self.kernel_size, padding = self.padding)
        self.conv12 = nn.Conv3d(in_channels = self.out2, out_channels = self.out2, kernel_size =self.kernel_size, padding = self.padding)
        
        self.convT4 = nn.ConvTranspose3d(in_channels = self.out2, out_channels = self.out1,kernel_size = 2,stride = 2)
        self.convT5 = nn.ConvTranspose3d(in_channels = 3*self.out1, out_channels = self.out1, kernel_size =self.kernel_size, stride = 1, padding = 0)
        
        self.conv13 = nn.Conv3d(in_channels = self.out1, out_channels = self.out1, kernel_size =self.kernel_size, padding = self.padding)
        self.conv14 = nn.Conv3d(in_channels = self.out1, out_channels = self.out1, kernel_size =self.kernel_size, padding = self.padding)

        self.conv15 = nn.Conv3d(in_channels = self.out1, out_channels = 3, kernel_size =self.kernel_size, padding = self.padding)


    def st_vec(self, x, scale):
      X = torch.zeros([batch_size,3,int(height*scale),int(width*scale),int(depth*scale)])
      X.cuda()
      for b in range(batch_size):
          s = structure_tensor_3d(x[b,0,:,:,:],1/2,1)
          val,vec = eig_special_3d(s)
          X[b,:,:,:,:] = torch.Tensor(vec)
      return X



    def forward(self, x):
       xstruct = self.down(x)
       s2 = self.st_vec(xstruct, scale)
       s1 = self.up(s2)
       s1 = s1.cuda()
       s2 = self.down2(s2)
       x = x.cuda()
      
       i1 = F.relu(self.I_conv1(x))
       s1 = F.relu(self.S_conv1(s1))
       i1 = F.relu(self.I_conv2(i1))
       i2 = self.Batchnorm1I(i1)
       s1 = F.relu(self.S_conv2(s1))
       s2 = self.Batchnorm1S(s1)
       i2 = self.I_mp1(i2)
       s2 = self.I_mp1(s2)


       i2 = F.relu(self.I_conv3(i2))
       s2 = F.relu(self.S_conv3(s2))
       i2 = F.relu(self.I_conv4(i2))
       s2 = F.relu(self.S_conv4(s2))
       i3 = self.Batchnorm2I(i2)
       s3 = self.Batchnorm2S(s2)
       i3 = self.I_mp2(i3)
       s3 = self.S_mp2(s3)

       i3 = F.relu(self.I_conv5(i3))
       s3 = F.relu(self.S_conv5(s3))
       i3 = F.relu(self.I_conv6(i3))
       i4 = self.Batchnorm3I(i3)
       s3 = F.relu(self.S_conv6(s3))
       s4 = self.Batchnorm3S(s3)
       i4 = self.I_mp3(i4)
       s4 = self.S_mp3(s4)

       i4 = F.relu(self.I_conv7(i4))
       s4 = F.relu(self.S_conv7(s4))
       i4 = F.relu(self.I_conv8(i4))
       s4 = F.relu(self.S_conv8(s4))
       s5 = self.Batchnorm4S(s4)
       i5 = self.Batchnorm4I(i4)
       i5 = self.I_mp4(i5)
       s5 = self.S_mp4(s5)

       
       i5 = F.relu(self.I_conv9(i5))
       s5 = F.relu(self.S_conv9(s5))
       i5 = F.relu(self.I_conv10(i5))
       s5 = F.relu(self.S_conv10(s5))

       

       uI4 = self.convIT1(i5)
       uS4 = self.convST1(s5)

       u4 = torch.cat((uI4,uS4, i4, s4),dim = 1)
       u4 = F.relu(self.conv7(u4))
       u4 = F.relu(self.conv8(u4))
       u3 = self.convT2(i4)
       u3 = torch.cat((u3,i3, s3),dim = 1)
       u3 = F.relu(self.conv9(u3))
       u3 = F.relu(self.conv10(u3))
       u2 = self.convT3(u3)
       u2 = torch.cat((u2,s2,i2),dim = 1)
       u2 = F.relu(self.conv11(u2))
       u2 = F.relu(self.conv12(u2))
       u1 = self.convT4(u2)
       u1 = torch.cat((u1,s1,i1), dim= 1)
       u1 = self.convT5(u1)
       u1 = F.relu(self.conv13(u1))
       u1 = F.relu(self.conv14(u1))
       u1 = self.conv15(u1)
       u1 = F.normalize(u1)
       return u1


class Net_3(nn.Module):
    def __init__(self):
        super(Net_3, self).__init__()
        self.out1 = 6*2
        self.out2 = 12*2
        self.out3 = 24*2
        self.out4 = 48*2
        self.out5 = 96*2


        self.kernel_size = 3
        self.padding = 1


        #StructureTensorPart
        self.up = nn.Upsample(scale_factor = 2, mode = 'trilinear',align_corners=True)
        self.down = nn.Upsample(scale_factor = 1/2,mode = 'trilinear',align_corners=True)
        self.down2 = nn.Upsample(scale_factor = 24/25,mode = 'trilinear',align_corners=True)
        self.S_conv1 = nn.Conv3d(in_channels = 2, out_channels = self.out1, kernel_size = self.kernel_size, padding = 0 )
        self.S_conv2 = nn.Conv3d(in_channels = self.out1, out_channels = self.out1, kernel_size =self.kernel_size, padding = self.padding)
        self.S_mp1 = nn.MaxPool3d(kernel_size = 2)
        self.S_bn1 = nn.BatchNorm3d(self.out1)
        self.S_conv3 = nn.Conv3d(in_channels = self.out1, out_channels = self.out2, kernel_size =self.kernel_size, padding = self.padding)
        self.S_conv4 = nn.Conv3d(in_channels = self.out2, out_channels = self.out2, kernel_size =self.kernel_size, padding = self.padding)
        self.S_mp2 = nn.MaxPool3d(kernel_size = 2)
        self.S_bn2 = nn.BatchNorm3d(self.out2)
        self.S_conv5 = nn.Conv3d(in_channels = self.out2, out_channels = self.out3, kernel_size = self.kernel_size, padding = self.padding)
        self.S_conv6 = nn.Conv3d(in_channels = self.out3, out_channels = self.out3, kernel_size = self.kernel_size, padding = self.padding)
        self.S_mp3 = nn.MaxPool3d(kernel_size = 2)
        self.S_bn3 = nn.BatchNorm3d(self.out3)
        self.S_conv7 = nn.Conv3d(in_channels = self.out3, out_channels = self.out4, kernel_size = self.kernel_size, padding = self.padding)
        self.S_conv8 = nn.Conv3d(in_channels = self.out4, out_channels = self.out4, kernel_size = self.kernel_size, padding = self.padding)
        self.S_mp4 = nn.MaxPool3d(kernel_size = 2)
        self.S_bn4 = nn.BatchNorm3d(self.out4)
        self.S_conv9 = nn.Conv3d(in_channels = self.out4, out_channels = self.out5, kernel_size = self.kernel_size, padding = self.padding)
        self.S_conv10 = nn.Conv3d(in_channels = self.out5, out_channels = self.out5, kernel_size = self.kernel_size, padding = self.padding)
        


        #FullImagePart
        self.I_conv1 = nn.Conv3d(in_channels = 1, out_channels = self.out1, kernel_size = self.kernel_size, padding = 0 )
        self.I_conv2 = nn.Conv3d(in_channels = self.out1, out_channels = self.out1, kernel_size =self.kernel_size, padding = self.padding)
        self.I_mp1 = nn.MaxPool3d(kernel_size = 2)
        self.I_bn1 = nn.BatchNorm3d(self.out1)
        self.I_conv3 = nn.Conv3d(in_channels = self.out1, out_channels = self.out2, kernel_size =self.kernel_size, padding = self.padding)
        self.I_conv4 = nn.Conv3d(in_channels = self.out2, out_channels = self.out2, kernel_size =self.kernel_size, padding = self.padding)
        self.I_mp2 = nn.MaxPool3d(kernel_size = 2)
        self.I_bn2 = nn.BatchNorm3d(self.out2)
        self.I_conv5 = nn.Conv3d(in_channels = self.out2, out_channels = self.out3, kernel_size = self.kernel_size, padding = self.padding)
        self.I_conv6 = nn.Conv3d(in_channels = self.out3, out_channels = self.out3, kernel_size = self.kernel_size, padding = self.padding)
        self.I_mp3 = nn.MaxPool3d(kernel_size = 2)
        self.I_bn3 = nn.BatchNorm3d(self.out3)
        self.I_conv7 = nn.Conv3d(in_channels = self.out3, out_channels = self.out4, kernel_size = self.kernel_size, padding = self.padding)
        self.I_conv8 = nn.Conv3d(in_channels = self.out4, out_channels = self.out4, kernel_size = self.kernel_size, padding = self.padding)
        self.I_mp4 = nn.MaxPool3d(kernel_size = 2)
        self.I_bn4 = nn.BatchNorm3d(self.out4)
        self.I_conv9 = nn.Conv3d(in_channels = self.out4, out_channels = self.out5, kernel_size = self.kernel_size, padding = self.padding)
        self.I_conv10 = nn.Conv3d(in_channels = self.out5, out_channels = self.out5, kernel_size = self.kernel_size, padding = self.padding)
        



        #Combining part

        self.convST1 = nn.ConvTranspose3d(in_channels = self.out5, out_channels = self.out4,kernel_size = 2, stride = 2)
        self.convIT1 = nn.ConvTranspose3d(in_channels = self.out5, out_channels = self.out4,kernel_size = 2, stride = 2)

        self.conv7 = nn.Conv3d(in_channels = 4*self.out4, out_channels = self.out4, kernel_size =self.kernel_size, padding = self.padding)
        self.conv8 = nn.Conv3d(in_channels = self.out4, out_channels = self.out4, kernel_size =self.kernel_size, padding = self.padding)
        
        self.convT2 = nn.ConvTranspose3d(in_channels = self.out4, out_channels = self.out3,kernel_size = 2,stride = 2)


        self.conv9 = nn.Conv3d(in_channels = 3 * self.out3, out_channels = self.out3, kernel_size =self.kernel_size, padding = self.padding)
        self.conv10 = nn.Conv3d(in_channels = self.out3, out_channels = self.out3, kernel_size =self.kernel_size, padding = self.padding)
        
        self.convT3 = nn.ConvTranspose3d(in_channels = self.out3, out_channels = self.out2, kernel_size = 2, stride = 2)
        
        self.conv11 = nn.Conv3d(in_channels = 3 * self.out2, out_channels = self.out2, kernel_size =self.kernel_size, padding = self.padding)
        self.conv12 = nn.Conv3d(in_channels = self.out2, out_channels = self.out2, kernel_size =self.kernel_size, padding = self.padding)
        
        self.convT4 = nn.ConvTranspose3d(in_channels = self.out2, out_channels = self.out1,kernel_size = 2,stride = 2)
        self.convT5 = nn.ConvTranspose3d(in_channels = 3*self.out1, out_channels = self.out1, kernel_size =self.kernel_size, stride = 1, padding = 0)
        
        self.conv13 = nn.Conv3d(in_channels = self.out1, out_channels = self.out1, kernel_size =self.kernel_size, padding = self.padding)
        self.conv14 = nn.Conv3d(in_channels = self.out1, out_channels = self.out1, kernel_size =self.kernel_size, padding = self.padding)

        self.conv15 = nn.Conv3d(in_channels = self.out1, out_channels = 2, kernel_size =self.kernel_size, padding = self.padding)
    def st_vec(self, x, scale):
      X = torch.zeros([batch_size,3,int(height*scale),int(width*scale),int(depth*scale)])
      X.cuda()
      for b in range(batch_size):
          s = structure_tensor_3d(x[b,0,:,:,:],1/2,1)
          val,vec = eig_special_3d(s)
          X[b,:,:,:,:] = torch.Tensor(vec)
      return X



    def forward(self, x):
       xstruct = self.down(x)
       s2 = self.st_vec(xstruct, scale)
       s1 = self.up(s2)
       s1 = cords2angles(s1)
       x = x.cuda()
      
       i1 = F.relu(self.I_conv1(x))
       s1 = F.relu(self.S_conv1(s1))
       i1 = F.relu(self.I_conv2(i1))
       s1 = F.relu(self.S_conv2(s1))
       i2 = self.I_bn1(i1)
       s2 = self.S_bn1(s1)
       i2 = self.I_mp1(i2)
       s2 = self.S_mp1(s2)


       i2 = F.relu(self.I_conv3(i2))
       s2 = F.relu(self.S_conv3(s2))
       i2 = F.relu(self.I_conv4(i2))
       s2 = F.relu(self.S_conv4(s2))
       i3 = self.I_bn2(i2)
       s3 = self.S_bn2(s2)
       i3 = self.I_mp2(i3)
       s3 = self.S_mp2(s3)

       i3 = F.relu(self.I_conv5(s3))
       s3 = F.relu(self.S_conv5(s3))
       i3 = F.relu(self.I_conv6(i3))
       s3 = F.relu(self.S_conv6(s3))
       i4 = self.I_bn3(i3)
       s4 = self.S_bn3(s3)
       i4 = self.I_mp3(i4)
       s4 = self.S_mp3(s4)

       i4 = F.relu(self.I_conv7(i4))
       s4 = F.relu(self.S_conv7(s4))
       i4 = F.relu(self.I_conv8(i4))
       s4 = F.relu(self.S_conv8(s4))
       i5 = self.I_bn4(i4)
       s5 = self.S_bn4(s4)
       i5 = self.I_mp4(i5)
       s5 = self.S_mp4(s5)

       
       i5 = F.relu(self.I_conv9(i5))
       s5 = F.relu(self.S_conv9(s5))
       i5 = F.relu(self.I_conv10(i5))
       s5 = F.relu(self.S_conv10(s5))

       

       uI4 = self.convIT1(i5)
       uS4 = self.convST1(s5)



       u4 = torch.cat((uI4,uS4, i4, s4),dim = 1)
       u4 = F.relu(self.conv7(u4))
       u4 = F.relu(self.conv8(u4))
       u3 = self.convT2(i4)
       u3 = torch.cat((u3,i3, s3),dim = 1)
       u3 = F.relu(self.conv9(u3))
       u3 = F.relu(self.conv10(u3))
       u2 = self.convT3(u3)
       u2 = torch.cat((u2,s2,i2),dim = 1)
       u2 = F.relu(self.conv11(u2))
       u2 = F.relu(self.conv12(u2))
       u1 = self.convT4(u2)
       u1 = torch.cat((u1,s1,i1), dim= 1)
       u1 = self.convT5(u1)
       u1 = F.relu(self.conv13(u1))
       u1 = F.relu(self.conv14(u1))
       u1 = torch.tanh(self.conv15(u1))*torch.pi

       #u1 = F.normalize(self.conv11(u1),dim = 1)
       return u1  
   
class Net_Tri_norm(nn.Module):
    def __init__(self):
        super(Net_Tri_norm, self).__init__()
        self.up = nn.Upsample(scale_factor = 2, mode = 'trilinear',align_corners=True)
        self.down = nn.Upsample(scale_factor = 1/2,mode = 'trilinear',align_corners=True)
    def st_vec(self,x):
     X = torch.zeros([batch_size,3,int(height*scale),int(width*scale),int(depth*scale)])
     for b in range(batch_size):
         s = structure_tensor_3d(x[b,0,:,:,:],1/2,1)
         val,vec = eig_special_3d(s)
         X[b,:,:,:,:] = torch.Tensor(vec)
     return X
    def forward(self, x):

       x = self.down(x)

       x = self.st_vec(x)

       x = self.up(x)

       x = F.normalize(x,dim = 1)
       return x

In [ ]:
modelinterpolation = Net_1()
modelinterpolation.eval()
modelinterpolation.cuda() 
modelcartesian = Net_2()
modelcartesian.load_state_dict(torch.load('CarthTRAINED-1.pt'))
modelcartesian.eval()
modelcartesian.cuda()

modelpolar = Net_3()
modelpolar.load_state_dict(torch.load('polarTRAINED_new2.pt'))
modelpolar.eval()
modelpolar.cuda()

Net_3(
  (up): Upsample(scale_factor=2.0, mode=trilinear)
  (down): Upsample(scale_factor=0.5, mode=trilinear)
  (down2): Upsample(scale_factor=0.96, mode=trilinear)
  (S_conv1): Conv3d(2, 12, kernel_size=(3, 3, 3), stride=(1, 1, 1))
  (S_conv2): Conv3d(12, 12, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (S_mp1): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (S_bn1): BatchNorm3d(12, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (S_conv3): Conv3d(12, 24, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (S_conv4): Conv3d(24, 24, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (S_mp2): MaxPool3d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (S_bn2): BatchNorm3d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (S_conv5): Conv3d(24, 48, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (S_conv6): Conv3d(48, 48, kernel_size=(3, 3, 3), stride=(1

In [ ]:
def CreateDataLoaders(split_ratios = [0.9,0.1],batch_size = 4,single = False):
    '''
    Loads saved the saved dataset, and returns dataloaders for the train, test
    and validation sets. Takes the parameter split_ratios for the data split,
    and batch_size to control batch size
    
    '''
    
    if sum(split_ratios) != 1:
        print("Sum of split_ratios must be 1!")
        raise ValueError
    
    X = np.load('rockwool_1_X.npy')
    Y = np.load('rockwool_1_Y.npy')
    
    #X = np.concatenate((X,X_2),axis = 0)  
    #Y = np.concatenate((Y,Y_2),axis = 0)
    nTraining,_,_,_ = X.shape
    tensor_y = torch.Tensor(Y)
    tensor_x = torch.Tensor(X)

    #tensor_precomp = torch.Tensor(precomp)
    my_dataset = TensorDataset(tensor_x,tensor_y)
    if single == True:
        return DataLoader(my_dataset, batch_size)
    
    # the data splitter needs lengths not ratios so we convert 
    lengths = [int(nTraining*split_ratios[i]) for i in range(2)]
    lengths[0] += nTraining - sum(lengths) 
    
    
    train,val = torch.utils.data.random_split(my_dataset,lengths,generator=torch.Generator().manual_seed(42))
    train = DataLoader(train, batch_size,shuffle = True)
    val = DataLoader(val, batch_size,shuffle = True)
    return train, val
scale = 0.5
height = width = depth = 50
batch_size = 16
dl = CreateDataLoaders(single = True,batch_size = batch_size)

KeyboardInterrupt: ignored

In [ ]:
height = width = depth = 50
criterion = nn.MSELoss()
# MSE and Time for 

Tinter = 0
Tinter1 = 0

Tstruct = 0
Tcart = 0
Tpol = 0
for Data in tqdm(dl):
        x_train,y_train = Data

        t0_inter = time.time()
        y_inter = modelinterpolation(x_train.reshape([batch_size,1,height,width,depth]))
        t1_inter = time.time()
        t0_inter1 = time.time()
        y_inter = modelinterpolation(x_train.reshape([batch_size,1,height,width,depth]))
        t1_inter1 = time.time()

        Tinter += t1_inter - t0_inter
        Tinter1 += t1_inter1 - t0_inter1
        
        






100%|██████████| 64/64 [00:55<00:00,  1.15it/s]


In [ ]:
print(Tinter)

27.491612195968628


In [ ]:

import scipy as scp


In [ ]:
def SaveThings(filename,vec):
  vec = vec[[2,1,0],:]

  # (Optional) Calculate linearity score - how signifcant the found direction is
  # Can be entered as weights parameter in visualization functions

  # (Optional) Flip opposite vectors to face the same direction - easier visualization
  flipOpposites = True
  flipMask = vec[0,:] < 0
  flipMask = np.array([flipMask,flipMask,flipMask])
  vec[flipMask] = -vec[flipMask]

  # Prepare mask - focus only on bright parts
  bgMask = I_cut < 60

  # Color representation of the directions in a volume
  # rgba = volume.convertToFan(vec, halfSphere=flipOpposites, weights=None, mask=bgMask) #Old method (new name)
  rgba = convertToIco(vec, weights=None, mask=bgMask) #New, bettter color scheme
  saveRgbaVolume(rgba,savePath=filename+".tiff")


  #%% alternative visualization - direction histogram visualized as a glyph

  sph = orientationVec(vec.reshape(3,-1), fullSphere=True, weights=None) 

  H, el, az, binArea = histogram2d(sph,bins=[100,200],norm='prob_binArea', weights=None)

  save_glyph(H,el,az,savePath=filename+".vtk", colorMap='Ico', flipColor=flipOpposites)
  return


In [ ]:
import numpy as np
import os
import tifffile
import scmap

#TODO: update documentation

def cart2sph(cart):
    """Performs a conversion from carthesian to sperical coordinates, mimics matlab function with the same name\n
    Source:https://stackoverflow.com/questions/4116658/faster-numpy-cartesian-to-spherical-coordinate-conversion\n
    Params:\n
    cart - (3,n) np.array with cartesian coordinates\n
    returns: (radius, elevation,azimuth)"""
    sph = np.zeros(cart.shape)
    xy = cart[0,:]**2 + cart[1,:]**2
    sph[0,:] = np.sqrt(xy + cart[2,:]**2)
    # sph[:,1] = np.arctan2(np.sqrt(xy), cart[2,:]) # for elevation angle defined from Z-axis down
    sph[1,:] = np.arctan2(cart[2,:], np.sqrt(xy)) # for elevation angle defined from XY-plane up
    sph[2,:] = np.arctan2(cart[1,:], cart[0,:])
    return sph

def orientationVec(vec,fullSphere=True, weights=None):
    """Get spherical orientation (azimuth, elevation) from collection of unit direction
    vectors mapped either onto a sphere or half sphere with a given poleOrder.\n
    Params:\n
    vec - (3,n) np.array with unit vectors\n
    fullSphere - if True, returns complete sphere (redundant but nice looking), if False, returns half sphere\n
    weights - additional weights related to the vectors, carried over in case they need to be doubled when transitioning to fullSphere
    """
    
    sph = cart2sph(vec)
    
    if fullSphere == False:
        # Flip directions onto half-sphere:
        idxN = np.where(sph[2,:] < -np.pi/2);
        idxP = np.where(sph[2,:] > np.pi/2);
        
        sph[2,idxN] = sph[2,idxN] + np.pi
        sph[1,idxN] = -sph[1,idxN]
        
        sph[2,idxP] = sph[2,idxP] - np.pi
        sph[1,idxP] = -sph[1,idxP]
    if fullSphere == True:
        sph2 = np.copy(sph)    
        
        sph2[1,:] = -sph2[1,:]
        
        idxN = np.where(sph2[2,:] < 0);
        idxP = np.where(sph2[2,:] > 0);
        
        sph2[2,idxN] = sph2[2,idxN] + np.pi
        sph2[2,idxP] = sph2[2,idxP] - np.pi
        
        sph = np.hstack((sph,sph2))
        if weights is not None:
            weights = np.hstack((weights,weights))
        
    if weights is None:
        return sph
    else:
        return sph, weights
    
def histogram2d(sph:np.array, bins=[100,200], norm=None, weights=None):
    """Wrapper to the numpy histogram 2d function with some extra steps.\n
    Params:\n
    sph - (3,n) spherical coordinates\n
    bins - number of histogram bins in each direction\n
    norm - normalization type:\n
        'prob' - normalizes to 0-1 range\n
        'binArea' - corrects for the variable bin area\n
        'prob_binArea' - combines both
    weights - additional weights for the histogram calculation (e.g. linearity score)
    """

    H,el,az = np.histogram2d(sph[1,:],sph[2,:],bins=bins, weights=weights) 

    az_diff = az[1:]-az[:-1]
    el_diff = np.sin(el[1:])-np.sin(el[:-1]) 
    el_dif_grid,az_dif_grid = np.meshgrid(el_diff,az_diff)
    binArea = (el_dif_grid*az_dif_grid).transpose()
    
    if norm=='prob':
        H = H/np.sum(H)
    elif norm=='binArea':
        H = H/binArea
    elif norm=='prob_binArea':
        H = H/binArea
        H = H/np.sum(H)
    else:
        assert norm is None, f"Wrong norm type: {norm}"

    return H, el, az, binArea

def save_glyph(H,el,az,savePath,colorMap=None,flipColor=True):
    """Creates glyph-like mesh visualization from the 2d spherical coordinate eigenvector histogram\n
    Params\n
    H - histogram values\n
    el, az - binning limits in elevation and azimuth direction\n
    savePath - path where the .vtk file of the glyph surface should be saved.\n
    normDiv - normalization value, what to divide the histogram values by\n
    colorMap - if not None, saves color information in a glyph using a chosen colormap. Possible options: 'Fan' or 'Ico'
    """

    el_center = (el[1:]+el[:-1])/2
    az_center = (az[1:]+az[:-1])/2
    el_center_grid,az_center_grid = np.meshgrid(el_center,az_center)
    x = np.cos(el_center_grid)*np.cos(az_center_grid)
    y = np.cos(el_center_grid)*np.sin(az_center_grid)
    z = np.sin(el_center_grid)
    XYZ = np.array([x,y,z])
    if colorMap is not None:
        XYZ_copy = np.copy(XYZ)
        if flipColor:
            # Flip to one half of the sphere, to have same colors on both sides
            flipMask = XYZ_copy[0,:] < 0
            flipMask = np.array([flipMask,flipMask,flipMask])
            XYZ_copy[flipMask] = -XYZ_copy[flipMask]
        if colorMap == 'Fan':
            RGB = convertToFan(np.expand_dims(XYZ_copy,-1),halfSphere=flipColor)[:3,:,:,0]
            RGB = 1-RGB
        elif colorMap == 'Ico':
            RGB = convertToIco(np.expand_dims(XYZ_copy,-1))[:3,:,:,0]
            RGB = 1-RGB
        else:
            RGB = None
    else:
        RGB = None
    XYZ = XYZ*H.T
    
    # Save as vtk surf
    assert os.path.dirname(savePath) == '' or os.path.exists(os.path.dirname(savePath)), f"Given path does not exist {savePath}"
    _, ext = os.path.splitext(savePath)
    assert ext == '.vtk', f"File extension ({ext}) is not .vtk"

    save_surf2vtk(savePath, XYZ, RGB)
    
    
def save_surf2vtk(filename, XYZ, RGB=None):
    '''  Writes a vtk file for a 3D surface with optional color information
    '''
    # Flip order to ZYX because paraview is weird
    XYZ = XYZ[[2,1,0],:]
    
    indices = np.arange(XYZ[0,:,:].size).reshape(XYZ[0,:,:].shape)
    vertices = np.moveaxis(np.reshape(XYZ,(3,-1)),0,-1)
    lu = indices[:-1,:-1]
    ru = indices[:-1,1:]
    rb = indices[1:,1:]
    lb = indices[1:,:-1]
    faces = np.c_[4*np.ones(lu.size), lu.ravel(), ru.ravel(), rb.ravel(), lb.ravel()]
    nf = faces.shape[0]

    if RGB is not None:
        colors = np.moveaxis(RGB.reshape((3,-1)),0,-1)
     
    with open(filename, 'w') as f:
        f.write('# vtk DataFile Version 3.0\n')
        f.write('saved from matlab using save_surf2vtk\n')
        f.write('ASCII\n')
        f.write('DATASET POLYDATA\n')
        f.write('POINTS {} float\n'.format(XYZ[0,:,:].size))
        np.savetxt(f, vertices, fmt='%.5g', newline='\n')
        f.write('POLYGONS {} {}\n'.format(nf,5*nf))
        np.savetxt(f, faces, fmt='%d', newline='\n')

        if RGB is not None:
            f.write('POINT_DATA {} \n'.format(XYZ[0,:,:].size))
            f.write('COLOR_SCALARS label 3\n')
            np.savetxt(f, colors, fmt='%.5g', newline='\n')
    

def hsv2rgb3d(vol):
    #https://www.had2know.org/technology/hsv-rgb-conversion-formula-calculator.html
    M = vol[2,:,:,:]
    m = M*(1-vol[1,:,:,:])
    mod_res = ((vol[0,:,:,:]*180/(np.pi*60)))%2
    z = (M-m)*(1-np.abs(mod_res-1))
    
    # Assuming H is in 0-180 range
    R = np.zeros_like(vol[0,:,:,:])
    G = np.zeros_like(vol[0,:,:,:])
    B = np.zeros_like(vol[0,:,:,:])
    
    Hrange1 = vol[0,:,:,:] < np.pi/3
    Hrange2 = np.bitwise_and(vol[0,:,:,:] >= np.pi/3, vol[0,:,:,:] < np.pi*2/3)
    Hrange3 = np.bitwise_and(vol[0,:,:,:] >= np.pi*2/3, vol[0,:,:,:] < np.pi)
    Hrange4 = np.bitwise_and(vol[0,:,:,:] >= np.pi, vol[0,:,:,:] < np.pi*4/3)
    Hrange5 = np.bitwise_and(vol[0,:,:,:] >= np.pi*4/3, vol[0,:,:,:] < np.pi*5/3)
    Hrange6 = vol[0,:,:,:] >= np.pi*5/3
    
    R[Hrange1] = M[Hrange1]
    G[Hrange1] = (z+m)[Hrange1]
    B[Hrange1] = m[Hrange1]

    R[Hrange2] = (z+m)[Hrange2]
    G[Hrange2] = M[Hrange2]
    B[Hrange2] = m[Hrange2]

    R[Hrange3] = m[Hrange3]
    G[Hrange3] = M[Hrange3]
    B[Hrange3] = (z+m)[Hrange3]
    
    R[Hrange4] = m[Hrange4]
    G[Hrange4] = (z+m)[Hrange4]
    B[Hrange4] = M[Hrange4]
    
    R[Hrange5] = (z+m)[Hrange5]
    G[Hrange5] = m[Hrange5]
    B[Hrange5] = M[Hrange5]
    
    R[Hrange6] = M[Hrange6]
    G[Hrange6] = m[Hrange6]
    B[Hrange6] = (z+m)[Hrange6]

    return np.array([R,G,B])

def convertToFan(vec, halfSphere=False, weights=None, mask=None):
    """Converts a volume of vectors to a volume of rgba values representing vector directions in a Fan scheme."""

    if halfSphere:
        # Stretch artificially the x values to use all colors
        fake_hsv = np.arctan2(vec[1,:,:],(vec[0,:,:]*2)-1)+np.pi
    else:
        fake_hsv = np.arctan2(vec[1,:,:],vec[0,:,:])+np.pi
    fake_hsv = np.array([fake_hsv,np.ones_like(fake_hsv),np.ones_like(fake_hsv)])

    fake_rgb = hsv2rgb3d(fake_hsv)
    colormap_vol = (1-vec[2,:,:]**2)*fake_rgb + 0.5*(vec[2,:,:]**2)

    colormap_vol = np.vstack((colormap_vol,np.ones_like(colormap_vol[0,:,:,:])[None,:]))

    if weights is not None:
        colormap_vol = colormap_vol*weights

    if mask is not None:
        mask_rgba = np.array((mask,mask,mask,mask))
        colormap_vol[mask_rgba] = 0

    return colormap_vol

def convertToIco(vec,  weights=None, mask=None):
    """Converts a volume of vectors to a volume of rgba values representing vector directions in a Icosahedron scheme."""

    coloring = scmap.Ico() 
    vec_flip = np.moveaxis(vec.reshape(3,-1), 0,-1)
    colormap_vol = coloring(vec_flip)
    colormap_vol = np.moveaxis(colormap_vol,0,-1).reshape(vec.shape)

    colormap_vol = np.vstack((colormap_vol,np.ones_like(colormap_vol[0,:,:,:])[None,:]))

    if weights is not None:
        colormap_vol = colormap_vol*weights

    if mask is not None:
        mask_rgba = np.array((mask,mask,mask,mask))
        colormap_vol[mask_rgba] = 0

    return colormap_vol

def saveRgbaVolume(rgba,savePath=None):
    
    rgba = np.moveaxis(rgba,0,-1)
    #  ParaView prefers to work with 8 bit uints
    rgba = (rgba*255).astype(np.uint8)
    # For some reason all colors are swapped in ParaView
    rgba = 255-rgba
    # ParaView needs a full range of values to work correctly
    rgba[0,0,0,3] = 255
    #Save as tiff
    if savePath is not None:
        assert os.path.dirname(savePath) == '' or os.path.exists(os.path.dirname(savePath)), f"Given path does not exist {savePath}"
        _, ext = os.path.splitext(savePath)
        assert ext == '.tiff', f"File extension ({ext}) is not .tiff"

        tifffile.imwrite(savePath, rgba)


def save_surf2vtk(filename, XYZ, RGB=None):
    '''  Writes a vtk file for a 3D surface with optional color information
    '''
    # Flip order to ZYX because paraview is weird
    XYZ = XYZ[[2,1,0],:]
    
    indices = np.arange(XYZ[0,:,:].size).reshape(XYZ[0,:,:].shape)
    vertices = np.moveaxis(np.reshape(XYZ,(3,-1)),0,-1)
    lu = indices[:-1,:-1]
    ru = indices[:-1,1:]
    rb = indices[1:,1:]
    lb = indices[1:,:-1]
    faces = np.c_[4*np.ones(lu.size), lu.ravel(), ru.ravel(), rb.ravel(), lb.ravel()]
    nf = faces.shape[0]

    if RGB is not None:
        colors = np.moveaxis(RGB.reshape((3,-1)),0,-1)
     
    with open(filename, 'w') as f:
        f.write('# vtk DataFile Version 3.0\n')
        f.write('saved from matlab using save_surf2vtk\n')
        f.write('ASCII\n')
        f.write('DATASET POLYDATA\n')
        f.write('POINTS {} float\n'.format(XYZ[0,:,:].size))
        np.savetxt(f, vertices, fmt='%.5g', newline='\n')
        f.write('POLYGONS {} {}\n'.format(nf,5*nf))
        np.savetxt(f, faces, fmt='%d', newline='\n')

        if RGB is not None:
            f.write('POINT_DATA {} \n'.format(XYZ[0,:,:].size))
            f.write('COLOR_SCALARS label 3\n')
            np.savetxt(f, colors, fmt='%.5g', newline='\n')
    


In [ ]:
x_train,y_train = next(iter(dl))
I_cut = x_train[0,:,:,:].numpy()

'''
y_inter = modelinterpolation(x_train.reshape([batch_size,1,height,width,depth]))
SaveThings('internew',y_inter[0,:,:,:,:].numpy())


s = structure_tensor_3d(x_train[0,:,:,:],1/2,1)
val,vec = eig_special_3d(s)
SaveThings('structnew',vec)
'''
'''
y_cart = modelcartesian(x_train.reshape([batch_size,1,height,width,depth]))
SaveThings('cartnew1',y_cart[0,:,:,:,:].cpu().detach().numpy())
'''


y_pol = modelcartesian(x_train.reshape([batch_size,1,height,width,depth]))
SaveThings('polnew2',y_pol[0,:,:,:,:].cpu().detach().numpy())




In [ ]:
import volume